In [ ]:
#|default_exp self-evaluating-datasets
import numpy as np
import matplotlib.pyplot as plt
import deepdish

import os
os.environ["GEOMSTATS_BACKEND"] = "pytorch"

# models
import torch

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#|export
from fastcore.all import *
import inspect
import pandas as pd

import sklearn
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt

def metric(func):
    setattr(func, 'tag', 'metric')
    return func

class Wrapper:
    def __init__(self, obj, **kwargs):
        self.obj = obj
        self.__dict__.update(kwargs)

class SelfEvaluatingDataset():
    def __init__(self,
                 datalist:List, # list of objects to be evaluated in the dataset. Usually includes multiple examples, e.g. a torus, sphere, saddle; multiple images, multiple validation datasets.
                 names:List, # names of the datasets in datalist.
                 dimension = 2, # Dimension of saddles and spheres
                 num_pointclouds = 100, # num pointclouds to make in total
                 num_points = 2000, # num points per pointclouds
                 noise_level = 0, # from 0 to 1. 1 is all noise.
                 include_planes = False, # if True, includes randomly sampled planes as a sanity check.
                ):
        store_attr()
        self.DS = [ # list of datasets
            Wrapper(obj, results={}, name=name) for obj, name in zip(datalist, names)
        ]
        self.idx = -1
    
    def __iter__(self):
        return self

    def __len__(self):
        return len(self.DS)
    
    def preprocess(self, unprocessed_data_object):
        return unprocessed_data_object # override

    def __next__(self):
        self.idx += 1
        if self.idx >= self.__len__():
            raise StopIteration
        result = self.DS[self.idx]
        return result

    def update(self,
               result,
               method_name='computed',
               ):
        """
        Store the result of the curvature computation by passing the computed curvature of the center (first) point.
        """
        self.DS[self.idx].results[method_name] = result

    def compute_metrics(self):
        self._aggregate_labels()
        metrics = self._get_metrics()
        self.metric_table = {}
        for metric in metrics:
            self.metric_table[metric.__name__] = {}
            for method_name in self.method_names:
                self.metric_table[metric.__name__][method_name] = self.compute(metric=metric, method_name=method_name)
        self.metric_table = pd.DataFrame(self.metric_table)
            
    def compute(self, metric, method_name):
        # Overwrite this class with your logic. It implements the computation of a single metric for a single method
        return metric(self.labels[method_name], self.labels['ground truth'])
    

    def _aggregate_labels(self):
        # returns a dictionary whose keys are method names, paired with a list of each of the results given by the metrics.
        # Just a more convenient data format for comparing method outputs.
        self.method_names = list(self.DS[0].results.keys())
        self.labels = {}
        for m in self.method_names:
            self.labels[m] = [self.DS[i].results[m] for i in range(self.__len__())]

    def plot(self, title = None):
        if title is None: title = f"In dimension {self.dimension}"
        # for each computed method on this dataset, we plot the histogram of saddles vs spheres
        self._aggregate_labels()
        # get the idxs for each type of dataset
        dataset_names = [self.DS.data_vars[i].attrs['name'] for i in range(len(self.DS))]
        unique_names = list(set(dataset_names))
        idxs_by_name = {n: [i for i, name in enumerate(dataset_names) if name == n] for n in unique_names}        
        for m in self.method_names: 
            if m != 'ks' and m != 'name':
                for dname in unique_names:
                    plt.hist(self.labels[m][idxs_by_name[dname]], bins=50, label = dname, edgecolor='none', linewidth=5)
                plt.legend()
                plt.xlabel(m)
                plt.title(title)
                plt.show()

    def table(self):
        self.compute_metrics()
        return self.metric_table

    def _get_metrics(self):
        tagged_functions = []
        for name, member in inspect.getmembers(self, predicate=inspect.ismethod):
            if hasattr(member, 'tag') and getattr(member, 'tag') == 'metric':
                tagged_functions.append(member)
        return tagged_functions

    @metric
    def pearson_r(self, a, b):
        return scipy.stats.pearsonr(a,b)
    

In [ ]:
dls = [3,1,4,1,5,9,2,6,5]
class Wrapper:
    def __init__(self, obj, **kwargs):
        self.obj = obj
        self.__dict__.update(kwargs)

# Example usage with a list of strings (but can be any objects)
original_objects = ["Apple", "Banana"]

wrapped_objects = [
    Wrapper(obj, color="Red", weight=150) for obj in original_objects
]

# Accessing attributes
print(wrapped_objects[0].obj)  # Output: Apple
print(wrapped_objects[0].color)  # Output: Red


Apple
Red


In [ ]:
wrapped_objects[1].__dict__['color']

'Red'